<a href="https://colab.research.google.com/github/hrumst/ML/blob/master/module_7_competition__%232.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Строим deep lab v3+ !
https://github.com/bonlime/keras-deeplab-v3-plus

In [0]:
%config IPComplater.greedy = True

In [0]:
# mkdir train2017 && unzip "./train2017.zip" -d train2017
# !wget http://images.cocodataset.org/zips/train2017.zip  
# !cp -r ./train2017 ./DRIVE/My\ Drive/competition_7/coco

In [0]:
from google.colab import drive

DRIVE_MOUNT = './DRIVE/'
DRIVE_MOUNT_SAVE_MODEL_PATH = 'My Drive/Colab Notebooks/competition_7/brand-logo-detection-dataset.zip'
drive.mount(DRIVE_MOUNT, force_remount=True)

In [0]:
import sys
sys.path.insert(0, './DRIVE/My Drive/competition_7/deeplab_v3_plus')

In [0]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

In [0]:
from model import Deeplabv3
deeplab_model = Deeplabv3(backbone='xception') 

In [0]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

# Generates labels using most basic setup.  Supports various image sizes.  Returns image labels in same format
# as original image.  Normalization matches MobileNetV2

trained_image_width=512
mean_subtraction_value=127.5

def get_mask(img_path):
    image = np.array(Image.open(img_path).convert('RGB'))

    # resize to max dimension of images from training dataset
    w, h, _ = image.shape
    ratio = float(trained_image_width) / np.max([w, h])
    resized_image = np.array(Image.fromarray(image.astype('uint8')).resize((int(ratio * h), int(ratio * w))))

    # apply normalization for trained dataset images
    resized_image = (resized_image / mean_subtraction_value) - 1.

    # pad array to square image to match training images
    pad_x = int(trained_image_width - resized_image.shape[0])
    pad_y = int(trained_image_width - resized_image.shape[1])
    resized_image = np.pad(resized_image, ((0, pad_x), (0, pad_y), (0, 0)), mode='constant')

    # make prediction
    res = deeplab_model.predict_on_batch(np.expand_dims(resized_image, 0))

    # print(res)
    labels = np.argmax(res.squeeze(), -1)

    labels = np.array(np.where(labels == 15, labels, 0))
    # remove padding and resize back to original image
    if pad_x > 0:
        labels = labels[:-pad_x]
    if pad_y > 0:
        labels = labels[:, :-pad_y]

    return np.array(Image.fromarray(labels.astype('uint8')).resize((h, w)))

In [0]:
import pandas as pd
# Из sample-submission читаем по каким именно картинкам нам нужно сделать предсказание
sample_submission = pd.read_csv('./DRIVE/My Drive/competition_7/coco/sample-submission.csv')
sample_submission.info()
sample_submission.head()

In [0]:
# кодирование маски в EncodedPixels
def mask_to_rle(mask):
    mask_flat = mask.flatten('F')
    flag = 0
    rle_list = list()
    for i in range(mask_flat.shape[0]):
        if flag == 0:
            if mask_flat[i] == 15:
                flag = 1
                starts = i+1
                rle_list.append(starts)
        else:
            if mask_flat[i] == 0:
                flag = 0
                ends = i
                rle_list.append(ends-starts+1)
    if flag == 1:
        ends = mask_flat.shape[0]
        rle_list.append(ends-starts+1)
    #sanity check
    if len(rle_list) % 2 != 0:
        print('NG')
    if len(rle_list) == 0:
        rle = np.nan
    else:
        rle = ' '.join(map(str,rle_list))
    return rle

In [0]:
%%time
from tqdm import tqdm
# Осталось дело за малым, 
# пройтись по списку картинок и сделать предикты с дальнейшим их кодированием в EncodedPixels
# Это займет значительное время...
submit_rle_arr = []

i = 0
for img_id in tqdm(sample_submission.ImageId.values):
    mask_out = get_mask(f'./DRIVE/My Drive/competition_7/coco/val2017/{img_id}')
    rle = mask_to_rle(mask_out)
    submit_rle_arr.append(rle)
    i += 1
    if i > 100:
        break

In [0]:

sample_submission['EncodedPixels'] = submit_rle_arr
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()